In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import os

/home/senne/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/home/senne/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/home/senne/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/home/senne/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/home/senne/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/home/senne/.local/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: cannot open shared object file: No such fi

11.3 Real time face detection

In [2]:
# Loading in the facetracking model
facetracker = load_model('facetracker_v2.h5')

In [3]:
# Function to save image from camera
def saveImage(capture, sample_coords, frame):
	SAVE_PATH =  os.path.join('input_image', 'input_image.jpg')
	x_min = (sample_coords[0]*450).astype(int) - 20
	y_min = (sample_coords[1]*450).astype(int) - 20
	x_max = (sample_coords[2]*450).astype(int) + 20
	y_max = (sample_coords[3]*450).astype(int) + 20

	h_array = [y_min, y_max]
	w_array = [x_min, x_max]

	h = np.diff(h_array)[0]
	w = np.diff(w_array)[0]

	cut = frame[y_min:y_min + h, x_min:x_min + w]
	
	cv2.imwrite(SAVE_PATH, cut)


In [8]:
# Capturing video frame
cap = cv2.VideoCapture(0)

while cap.isOpened():
    _ , frame = cap.read()
    # Cutting frame down to augmented size
    frame = frame[50:500, 50:500,:]
    
    # Change from BGR tot RGB
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Resizing to 120 x 120
    resized = tf.image.resize(rgb, (120,120))

    # Scaling down and give it to the .predict() function of the facetracker
    yhat = facetracker.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]

    # Declaring waitkey
    k = cv2.waitKey(1)

    if yhat[0] > 0.5:
        # If pressed 's', and face is detected --> take cutted snapshot
        if k == ord('s'):
            saveImage(cap, sample_coords, frame)
            print('s')
            
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 
                            (0,255,0), 2)

        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [80,0])), 
                            (0,255,0), -1)

        # Controls the text rendered
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                               [0,-5])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

    cv2.imshow('FaceTrack', frame)

    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 2s 2s/step
Running
1/1 [==============================] - 2s 2s/step
Running
1/1 [==============================] - 2s 2s/step
Running
1/1 [==============================] - 2s 2s/step
Running
1/1 [==============================] - 2s 2s/step
Running
1/1 [==============================] - 2s 2s/step
Running
1/1 [==============================] - 2s 2s/step
Running
1/1 [==============================] - 2s 2s/step
Running
1/1 [==============================] - 2s 2s/step
Running
1/1 [==============================] - 1s 1s/step
Running
1/1 [==============================] - 2s 2s/step
Running
1/1 [==============================] - 2s 2s/step
Running
1/1 [==============================] - 2s 2s/step
Running
1/1 [==============================] - 2s 2s/step
Running
1/1 [==============================] - 2s 2s/step
Running
1/1 [==============================] - 2s 2s/step
Running
1/1 [==============================] - 2s 2s/step
Running
1/1 [=========

KeyboardInterrupt: 